# Implementing Recommender Systems - Lab

## Introduction

In this lab, you'll practice creating a recommender system model using `surprise`. You'll also get the chance to create a more complete recommender system pipeline to obtain the top recommendations for a specific user.


## Objectives

In this lab you will: 

- Use surprise's built-in reader class to process data to work with recommender algorithms 
- Obtain a prediction for a specific user for a particular item 
- Introduce a new user with rating to a rating matrix and make recommendations for them 
- Create a function that will return the top n recommendations for a user 


For this lab, we will be using the famous 1M movie dataset. It contains a collection of user ratings for many different movies. In the last lesson, you were exposed to working with `surprise` datasets. In this lab, you will also go through the process of reading in a dataset into the `surprise` dataset format. To begin with, load the dataset into a Pandas DataFrame. Determine which columns are necessary for your recommendation system and drop any extraneous ones.

In [1]:
import pandas as pd
df = pd.read_csv('./ml-latest-small/ratings.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100836 entries, 0 to 100835
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 3.1 MB


In [2]:
# Drop unnecessary columns
new_df = df.drop('timestamp', axis=1)

It's now time to transform the dataset into something compatible with `surprise`. In order to do this, you're going to need `Reader` and `Dataset` classes. There's a method in `Dataset` specifically for loading dataframes.

In [3]:
new_df.rating.describe()

count    100836.000000
mean          3.501557
std           1.042529
min           0.500000
25%           3.000000
50%           3.500000
75%           4.000000
max           5.000000
Name: rating, dtype: float64

In [4]:
from surprise import Reader, Dataset
# read in values as Surprise dataset 
reader = Reader(rating_scale=(0.5, 5))
data = Dataset.load_from_df(new_df, reader)


Let's look at how many users and items we have in our dataset. If using neighborhood-based methods, this will help us determine whether or not we should perform user-user or item-item similarity

In [5]:
dataset = data.build_full_trainset()
print('Number of users: ', dataset.n_users, '\n')
print('Number of items: ', dataset.n_items)

Number of users:  610 

Number of items:  9724


## Determine the best model 

Now, compare the different models and see which ones perform best. For consistency sake, use RMSE to evaluate models. Remember to cross-validate! Can you get a model with a higher average RMSE on test data than 0.869?

In [6]:
# importing relevant libraries
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import SVD
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline
from surprise.model_selection import GridSearchCV
import numpy as np

In [7]:
## Perform a gridsearch with SVD
# ⏰ This cell may take several minutes to run
param_grid_svd =  {'n_factors':[20, 100], 
                   'n_epochs': [5, 10], 
                   'lr_all': [0.002, 0.005], 
                   'reg_all': [0.4, 0.6]}
gs_svd = GridSearchCV(SVD, param_grid=param_grid_svd, n_jobs=-1, joblib_verbose=5)
gs_svd.fit(data)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   34.8s
[Parallel(n_jobs=-1)]: Done  64 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed:  4.4min finished


In [8]:
# print out optimal parameters for SVD after GridSearch
gs_svd.best_params

{'rmse': {'n_factors': 20, 'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4},
 'mae': {'n_factors': 20, 'n_epochs': 10, 'lr_all': 0.005, 'reg_all': 0.4}}

In [9]:
gs_svd.best_score

{'rmse': 0.8906020106276541, 'mae': 0.6892402711529061}

In [10]:
# cross validating with KNNBasic
sim_cos = {'name': 'cosine', 'user_based':True}
basic = KNNBasic(sim_options=sim_cos)
knn_cv = cross_validate(basic, data, n_jobs=-1)

In [11]:
# print out the average RMSE score for the test set
print(f"ave RMSE for test set {knn_cv['test_rmse'].mean()}")

ave RMSE for test set 0.9729717172419665


In [12]:
# cross validating with KNNBaseline
baseline = KNNBaseline(sim_options=sim_cos)
knn_baseline_cv = cross_validate(baseline, data, n_jobs=-1, verbose=True)

Evaluating RMSE, MAE of algorithm KNNBaseline on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8758  0.8916  0.8682  0.8805  0.8778  0.8788  0.0076  
MAE (testset)     0.6691  0.6802  0.6676  0.6738  0.6724  0.6726  0.0044  
Fit time          0.63    0.68    0.72    0.69    0.60    0.66    0.05    
Test time         3.53    3.57    3.51    3.50    3.13    3.45    0.16    


In [13]:
# print out the average score for the test set
print(f"ave RMSE for test set {knn_baseline_cv['test_rmse'].mean()}")

ave RMSE for test set 0.8787740206098953


Based off these outputs, it seems like the best performing model is the SVD model with `n_factors = 50` and a regularization rate of 0.05. Use that model or if you found one that performs better, feel free to use that to make some predictions.

## Making Recommendations

It's important that the output for the recommendation is interpretable to people. Rather than returning the `movie_id` values, it would be far more valuable to return the actual title of the movie. As a first step, let's read in the movies to a dataframe and take a peek at what information we have about them.

In [14]:
df_movies = pd.read_csv('./ml-latest-small/movies.csv')

In [15]:
df_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


## Making simple predictions
Just as a reminder, let's look at how you make a prediction for an individual user and item. First, we'll fit the SVD model we had from before.

In [16]:
svd = SVD(n_factors= 50, reg_all=0.05)
svd.fit(dataset)

In [17]:
svd.predict(2, 4)

Prediction(uid=2, iid=4, r_ui=None, est=2.959355918965109, details={'was_impossible': False})

This prediction value is a tuple and each of the values within it can be accessed by way of indexing. Now let's put our knowledge of recommendation systems to do something interesting: making predictions for a new user!

## Obtaining User Ratings 

It's great that we have working models and everything, but wouldn't it be nice to get to recommendations specifically tailored to your preferences? That's what we'll be doing now. The first step is to create a function that allows us to pick randomly selected movies. The function should present users with a movie and ask them to rate it. If they have not seen the movie, they should be able to skip rating it. 

The function `movie_rater()` should take as parameters: 

* `movie_df`: DataFrame - a dataframe containing the movie ids, name of movie, and genres
* `num`: int - number of ratings
* `genre`: string - a specific genre from which to draw movies

The function returns:
* rating_list : list - a collection of dictionaries in the format of {'userId': int , 'movieId': int , 'rating': float}

#### This function is optional, but fun :) 

In [18]:
def movie_rater(movie_df, num, userid, genre=None):
    ratings = []
    if genre != None:
        movies_to_rate = movie_df[movie_df.genres.str.contains(genre)].sample(num)
    else:
        movies_to_rate = movie_df.sample(num)
    display(movies_to_rate)
    for i in range(num):
        print('How do you rate this movie on a scale of 1-5, press n if you have not seen :')
        print(movies_to_rate.iloc[i]['title'])
        rating = input()
        if rating == 'n':
            rating = 0
        ratings.append({'userId': userid, 
                        'movieId': movies_to_rate.iloc[i]['movieId'],
                       'rating': float(rating)})
    return ratings
        

In [21]:
user_rating = movie_rater(df_movies, 10, 1000, 'Sci')

,movieId,title,genres
5685,27728,Ghost in the Shell 2: Innocence (a.k.a. Innoce...,Action|Animation|Drama|Sci-Fi|Thriller
8048,98607,Redline (2009),Action|Animation|Sci-Fi
3682,5072,Metropolis (2001),Animation|Sci-Fi
9346,160978,Hellevator (2004),Horror|Sci-Fi
6706,58404,Justice League: The New Frontier (2008),Action|Adventure|Animation|Fantasy|Sci-Fi
7607,86864,Mothra (Mosura) (1961),Adventure|Fantasy|Sci-Fi
902,1200,Aliens (1986),Action|Adventure|Horror|Sci-Fi
2316,3070,Adventures of Buckaroo Banzai Across the 8th D...,Adventure|Comedy|Sci-Fi
1970,2615,My Science Project (1985),Adventure|Sci-Fi
7438,80969,Never Let Me Go (2010),Drama|Romance|Sci-Fi


How do you rate this movie on a scale of 1-5, press n if you have not seen :
Ghost in the Shell 2: Innocence (a.k.a. Innocence) (Inosensu) (2004)
n
How do you rate this movie on a scale of 1-5, press n if you have not seen :
Redline (2009)
n
How do you rate this movie on a scale of 1-5, press n if you have not seen :
Metropolis (2001)
5
How do you rate this movie on a scale of 1-5, press n if you have not seen :
Hellevator (2004)
n
How do you rate this movie on a scale of 1-5, press n if you have not seen :
Justice League: The New Frontier (2008) 
n
How do you rate this movie on a scale of 1-5, press n if you have not seen :
Mothra (Mosura) (1961)
4
How do you rate this movie on a scale of 1-5, press n if you have not seen :
Aliens (1986)
4
How do you rate this movie on a scale of 1-5, press n if you have not seen :
Adventures of Buckaroo Banzai Across the 8th Dimension, The (1984)
3
How do you rate this movie on a scale of 1-5, press n if you have not seen :
My Science Project (1985)


In [22]:
user_rating

[{'userId': 1000, 'movieId': 27728, 'rating': 0.0},
 {'userId': 1000, 'movieId': 98607, 'rating': 0.0},
 {'userId': 1000, 'movieId': 5072, 'rating': 5.0},
 {'userId': 1000, 'movieId': 160978, 'rating': 0.0},
 {'userId': 1000, 'movieId': 58404, 'rating': 0.0},
 {'userId': 1000, 'movieId': 86864, 'rating': 4.0},
 {'userId': 1000, 'movieId': 1200, 'rating': 4.0},
 {'userId': 1000, 'movieId': 3070, 'rating': 3.0},
 {'userId': 1000, 'movieId': 2615, 'rating': 0.0},
 {'userId': 1000, 'movieId': 80969, 'rating': 0.0}]

If you're struggling to come up with the above function, you can use this list of user ratings to complete the next segment

In [38]:
user_rating

[{'userId': 1000, 'movieId': 55245, 'rating': '5'},
 {'userId': 1000, 'movieId': 2491, 'rating': '4'},
 {'userId': 1000, 'movieId': 4718, 'rating': '4'},
 {'userId': 1000, 'movieId': 5990, 'rating': '3'}]

### Making Predictions With the New Ratings
Now that you have new ratings, you can use them to make predictions for this new user. The proper way this should work is:

* add the new ratings to the original ratings DataFrame, read into a `surprise` dataset 
* train a model using the new combined DataFrame
* make predictions for the user
* order those predictions from highest rated to lowest rated
* return the top n recommendations with the text of the actual movie (rather than just the index number) 

In [24]:
## add the new ratings to the original ratings DataFrame
new_ratings_df = new_df.append(user_rating, ignore_index=True)

# create a new dataset from the new dataframe, and use the reader created at the beginning
new_data = Dataset.load_from_df(new_ratings_df, reader)

In [25]:
# train a model using the new combined DataFrame
new_svd = SVD(n_factors= 50, reg_all=0.05)
new_svd.fit(new_data.build_full_trainset())

In [29]:
new_svd.predict(1000, 1200)

Prediction(uid=1000, iid=1200, r_ui=None, est=3.0863194921534576, details={'was_impossible': False})

In [30]:
# make predictions for the user
# you'll probably want to create a list of tuples in the format (movie_id, predicted_score)
movie_list = []
for movie_id in new_ratings_df.movieId.unique():
    prediction = new_svd.predict(1000, movie_id)[3]
    movie_list.append((movie_id, prediction))

In [32]:
# order the predictions from highest to lowest rated

ranked_movies = sorted(movie_list, key=lambda x: x[1], reverse=True)
ranked_movies[:10]

[(318, 3.386810582514329),
 (750, 3.261226330666244),
 (1276, 3.2577471573862233),
 (527, 3.25640859438533),
 (260, 3.2299611183391246),
 (858, 3.2215232476627107),
 (3451, 3.2214791386223287),
 (1104, 3.2158495500822877),
 (1204, 3.1855057167831533),
 (741, 3.1820129871813996)]

 For the final component of this challenge, it could be useful to create a function `recommended_movies()` that takes in the parameters:
* `user_ratings`: list - list of tuples formulated as (user_id, movie_id) (should be in order of best to worst for this individual)
* `movie_title_df`: DataFrame 
* `n`: int - number of recommended movies 

The function should use a `for` loop to print out each recommended *n* movies in order from best to worst

In [35]:
# return the top n recommendations using the 
def recommended_movies(user_ratings,movie_title_df,n):
    for i in range(n):
        movie_title = movie_title_df[movie_title_df['movieId'] == user_ratings[i][0]]['title']
        print(f'Recommendation #{i+1}: {movie_title}')
            
recommended_movies(ranked_movies,df_movies,10)

Recommendation #1: 277    Shawshank Redemption, The (1994)
Name: title, dtype: object
Recommendation #2: 602    Dr. Strangelove or: How I Learned to Stop Worr...
Name: title, dtype: object
Recommendation #3: 975    Cool Hand Luke (1967)
Name: title, dtype: object
Recommendation #4: 461    Schindler's List (1993)
Name: title, dtype: object
Recommendation #5: 224    Star Wars: Episode IV - A New Hope (1977)
Name: title, dtype: object
Recommendation #6: 659    Godfather, The (1972)
Name: title, dtype: object
Recommendation #7: 2582    Guess Who's Coming to Dinner (1967)
Name: title, dtype: object
Recommendation #8: 841    Streetcar Named Desire, A (1951)
Name: title, dtype: object
Recommendation #9: 906    Lawrence of Arabia (1962)
Name: title, dtype: object
Recommendation #10: 596    Ghost in the Shell (Kôkaku kidôtai) (1995)
Name: title, dtype: object


## Level Up (Optional)

* Try and chain all of the steps together into one function that asks users for ratings for a certain number of movies, then all of the above steps are performed to return the top $n$ recommendations
* Make a recommender system that only returns items that come from a specified genre

## Summary

In this lab, you got the chance to implement a collaborative filtering model as well as retrieve recommendations from that model. You also got the opportunity to add your own recommendations to the system to get new recommendations for yourself! Next, you will learn how to use Spark to make recommender systems.